In [1]:
from deephaven_server import Server

my_dh_key = "Ja080104"

s = Server(jvm_args=[f"-Dauthentication.psk={my_dh_key}", "-Xmx4g"], port=3000).start()

# Starting io.deephaven.python.server.EmbeddedServer
deephaven.cacheDir=/home/jasleen/.cache/deephaven
deephaven.configDir=/home/jasleen/.config/deephaven
deephaven.dataDir=/home/jasleen/.local/share/deephaven
# io.deephaven.internal.log.LoggerFactoryServiceLoaderImpl: searching for 'io.deephaven.internal.log.LoggerFactory'...
# io.deephaven.internal.log.LoggerFactoryServiceLoaderImpl: found 'io.deephaven.internal.log.LoggerFactorySlf4j'
Server started on port 3000


2023-09-12 09:51:05.301449: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-12 09:51:05.716861: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-12 09:51:05.718994: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-12 09:51:07.510101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


deephaven.table.Table(io.deephaven.engine.table.Table(objectRef=0x7fada8f2be40, num_rows = 10729, columns = {'Toke...}))


In [8]:
from deephaven import read_csv
import pandas as pd
import deephaven as dh

sentiment = read_csv("https://media.githubusercontent.com/media/deephaven/examples/main/Sentiment/csv/sentiment.csv")
sentiment = sentiment.view(["text", "sentiment"]).where("sentiment != 'Neutral'")
sentiment

deephaven.table.Table(io.deephaven.engine.table.Table(objectRef=0x5327188, num_rows = 13871, columns = {'text': ja...}))

In [40]:
# from deephaven import dataFrameToTable, tableToDataFrame
from deephaven import pandas as dhpd
from deephaven_ipywidgets import DeephavenWidget

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import re

def update_text(text):
    return re.sub("[^a-zA-z0-9\s]", "", text.lower()).replace("rt", " ")

sentiment = sentiment.update("text = update_text(text)")
df_sentiment = dhpd.to_pandas(sentiment)
# print(df_sentiment)
# df_sentiment = tableToDataFrame(sentiment)

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=" ")
tokenizer.fit_on_texts(df_sentiment["text"].values)

x = tokenizer.texts_to_sequences(df_sentiment["text"].values)
x = pad_sequences(x)
y = pd.get_dummies(df_sentiment["sentiment"]).values

data_tokenized = np.hstack((x, y))
# print(data_tokenized.shape)

x_col_names = ["Tokenized_" + str(i) for i in range(x.shape[1])]
y_col_names = ["Negative", "Positive"]
col_names = x_col_names + y_col_names
# print(len(col_names))

df_tokenized = pd.DataFrame(data_tokenized, columns=col_names)
tokenized_sentiment = dhpd.to_table(df_tokenized)
print(tokenized_sentiment)

deephaven.table.Table(io.deephaven.engine.table.Table(objectRef=0x9ed3fc8, num_rows = 13871, columns = {'Tokenized...}))

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

embed_dim = 128
lstm_out = 196
max_features = 2000

sa_model = Sequential()
sa_model.add(Embedding(max_features, embed_dim, input_length=x.shape[1]))
sa_model.add(SpatialDropout1D(0.4))
sa_model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
sa_model.add(Dense(2, activation="softmax"))

sa_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print(sa_model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d_1 (Spati  (None, 28, 128)           0         
 alDropout1D)                                                    
                                                                 
 lstm_1 (LSTM)               (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [51]:
split_pct = 67

def split_train_test(x, y, split_percentage):
    random_seeds = np.random.rand(x.shape[0])
    split = random_seeds < np.percentile(random_seeds, split_percentage)
    return x[split], x[~split], y[split], y[~split]

x_train, x_test, y_train, y_test = split_train_test(x, y, split_pct)
train_table = dhpd.to_table(pd.DataFrame(np.hstack((x_train, y_train)), columns=col_names))
test_table = dhpd.to_table(pd.DataFrame(np.hstack((x_test, y_test)), columns=col_names))

print(f"No of rows in train table: {train_table.size}")
print(f"No of rows in test table: {test_table.size}")


No of rows in train table: 9293
No of rows in test table: 4578


In [47]:
from deephaven.learn import gather
from deephaven import learn

def table_to_numpy_int(rows, colums):
    return gather.table_to_numpy_2d(rows, columns, dtype=np.intc)

def numpy_to_table(data, index):
    return data 


In [ ]:
def train_model(data):
    global sa_model
    x_train = data[:, :-2]
    y_train = data[:, -2:]
    sa_model.fit(x_train, y_train, epochs = 7, batch_size = 32, verbose = 2)

learn.learn(
    table = train_table,
    model_func = train_model,
    inputs = [learn.Input(col_names, table_to_numpy_int)],
    outputs = None,
    batch_size = train_table.size
)


Epoch 1/7


DHError: failed to complete the learn function. : java.lang.RuntimeException: Error in Python interpreter:
Traceback (most recent call last):
  File "/home/jasleen/Desktop/Thapar/Conversational-Systems/venv/lib/python3.11/site-packages/deephaven/table.py", line 835, in update
    return Table(j_table=self.j_table.update(*formulas))
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: io.deephaven.engine.table.impl.select.FormulaEvaluationException: In formula: __Result = __FutureOffset.getFuture().get()
	at io.deephaven.temp.c_4c37b714b919339551dc4142f75f4bec0e9b15aefafb4799ff74a07d2d8e8819v55_0.Formula.applyFormulaPerItem(Formula.java:153)
	at io.deephaven.temp.c_4c37b714b919339551dc4142f75f4bec0e9b15aefafb4799ff74a07d2d8e8819v55_0.Formula.lambda$fillChunkHelper$4(Formula.java:142)
	at io.deephaven.engine.rowset.RowSequence.lambda$forAllRowKeys$0(RowSequence.java:175)
	at io.deephaven.engine.rowset.impl.singlerange.SingleRangeMixin.forEachRowKey(SingleRangeMixin.java:

Epoch 1/7
225/225 - 26s - loss: 0.4312 - accuracy: 0.8208 - 26s/epoch - 117ms/step
Epoch 2/7
225/225 - 24s - loss: 0.3007 - accuracy: 0.8724 - 24s/epoch - 106ms/step
Epoch 3/7
225/225 - 26s - loss: 0.2493 - accuracy: 0.8966 - 26s/epoch - 115ms/step
Epoch 4/7


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from deephaven import pandas as dhpd
from deephaven.learn import gather
from deephaven import learn
from deephaven import read_csv
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import re

sentiment = read_csv("https://media.githubusercontent.com/media/deephaven/examples/main/Sentiment/csv/sentiment.csv")
sentiment = sentiment.view(["text", "sentiment"]).where("sentiment != 'Neutral'")

def update_text(text):
    return re.sub("[^a-zA-z0-9\s]", "", text.lower()).replace("rt", " ")

sentiment = sentiment.update("text = update_text(text)")
df_sentiment = dhpd.to_pandas(sentiment)

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=" ")
tokenizer.fit_on_texts(df_sentiment["text"].values)
x = tokenizer.texts_to_sequences(df_sentiment["text"].values)
x = pad_sequences(x)
y = pd.get_dummies(df_sentiment["sentiment"]).values

data_tokenized = np.hstack((x, y))

x_col_names = ["Tokenized_" + str(i) for i in range(x.shape[1])]
y_col_names = ["Negative", "Positive"]
col_names = x_col_names + y_col_names

embed_dim = 128
lstm_out = 196  
max_features = 2000

sa_model = Sequential()
sa_model.add(Embedding(max_features, embed_dim, input_length=x.shape[1]))
sa_model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
sa_model.add(Dense(2, activation="softmax"))
sa_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

split_pct = 67

def split_train_test(x, y, split_percentage):
    random_seeds = np.random.rand(x.shape[0])
    split = random_seeds < np.percentile(random_seeds, split_percentage)
    return x[split], x[~split], y[split], y[~split]

x_train, x_test, y_train, y_test = split_train_test(x, y, split_pct)
train_table = dhpd.to_table(pd.DataFrame(np.hstack((x_train, y_train)), columns=col_names))
test_table = dhpd.to_table(pd.DataFrame(np.hstack((x_test, y_test)), columns=col_names))

def table_to_numpy_int(rows, columns):
    return gather.table_to_numpy_2d(rows, columns)

def numpy_to_table(data, index):
    return data

def train_model(data):
    global sa_model
    x_train = data[:, :-2]
    y_train = data[:, -2:]
    sa_model.fit(x_train, y_train, epochs=7, batch_size=32, verbose=2)

learn.learn(
    table = train_table,
    model_func = train_model,
    inputs = [learn.Input(col_names, table_to_numpy_int)],
    outputs = None,
    batch_size = train_table.size
)

